In [1]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys

def SecModelParameter ( VarsValue, m, n, my_rhs, my_coef, BranchedVariables):
#     print 'BranchedVariables',BranchedVariables
    unbranched=list()
    c=[]
    d=(np.zeros(((2 * m) + (4 * n), 2*n)))
    halatest=[]
    a=[]
    newrow=[]
    zer=(np.zeros(((2 * m) + (4 * n), 2*n)))
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)
    NodeNum=n-len(unbranched)
    
    
    for i in unbranched:   
        for h in range (m):
            c.append(0)
        for h in range (m):
            lin=0            
            for j in range (n):
#                 print 'i', i, 'h', h,'j',j
#                 print 'j1',j
                if j in (BranchedVariables):
#                     print 'j2',j
                    sv = VarsValue[BranchedVariables.index(j)]
#                     print 'sv', sv
                    lin =lin+ (sv * my_coef[h][j]) 
#                     print 'lin', lin
#                 print 'sv'
            lin=lin- my_rhs[h]
            c.append(lin)                
        for h in range (n):
            c.append(0)
        for h in range (n):
            c.append(0)

        for jj in range (n):
            lin2=-1
            if jj in (BranchedVariables):
                sv = VarsValue[BranchedVariables.index(jj)]
                lin2 =lin2+ sv
            c.append(lin2)
            
        for jj in range (n):
            lin2=0
            if jj in (BranchedVariables):
                lin2 =-1*VarsValue[BranchedVariables.index(jj)]
            c.append(lin2)

              
    k=NodeNum-1 

    for i in unbranched: 
        k=k+1
        for t in range(m):              
            for j in range(n):    
                d[(k-NodeNum)*(2*m+4*n)+t][j+n]=my_coef[t][j] 
                
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t][n+i]=d[(k-NodeNum)*(2*m+4*n)+t][n+i]-my_rhs[t]   
                
        for t in range(m):  
            for kk in unbranched:
                d[(k-NodeNum)*(2*m+4*n)+t+m][kk]=my_coef[t][kk] 
                

        for t in range(m):  
            for j in range(n):
                d[(k-NodeNum)*(2*m+4*n)+t+m][j+n]=-1*my_coef[t][j] 
                 
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]=d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]+my_rhs[t] 
                
        for t in range(n):              
            d[(k-NodeNum)*(2*m+4*n)+t+2*m][i]=-1 
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+t][n+t]=1 
        
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+n+t][n+t]=-1 
        
        for mm in unbranched: 
             
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+mm][mm]=1  
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]=d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]+1            
            

        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+t][n+t]=-1 

        for t in unbranched:  
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][t]=-1  
               
        
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][n+t]=1 
            
        d=np.vstack([d, zer])                          
            
    d = np.delete(d, slice(((n - NodeNum)*((2 * m) + (4 * n))),((n-NodeNum+1)*((2 * m) + (4 * n)))), axis=0)
#     print 'c',c
#     print 'unbranched',unbranched
    return c, d,unbranched
#     print c, d,unbranched

In [2]:
def SecondModel (cc,dd,NotBranched):
#     print 'cc,dd,NotBranched',cc,dd,NotBranched
    numrows = len(dd)    # 3 rows in your example
    numcols = len(dd[0])
    VarNum=numcols/2
    ConstNum=numrows/len(NotBranched)
    my_obj=[]
    my_ub=[]
    my_lb=[]
    my_colnames=[]
    my_rownames=["c1"]
    my_coef=[] 
    my_row=[]
    my_rhs=[]
    my_sense=""
    my_Nmah=list()
    
    for i in range(numrows):
        my_obj=cc
        my_ub.append(1)
        my_lb.append(0)
        my_colnames.append("y"+str(i))

    sumarray=[]     
    NotBranched.sort()    
#     sumshode=list()
    for t in range (VarNum):
        sumlist=list()
        for j in range(numrows):
#             print 't,j',t,j
            if t in NotBranched: 
                indj=NotBranched.index(t)
#                 print 'indj',indj
                indjrange=range(indj*ConstNum,(indj+1)*ConstNum)
#                 print 'indjrange',indjrange
                if j in indjrange:
#                     print 'dd[j,t]+dd[j,t+VarNum]',j,'ll',dd[j,t]+dd[j,t+VarNum]
                    sumlist.append(dd[j,t]+dd[j,t+VarNum])
                    dd[j,t+VarNum]=0
                else:
                    sumlist.append(dd[j,t])
            else:
                sumlist.append(dd[j,t])
        sumarray.append(sumlist)
#     print 'sumarray',sumarray
#     print 'newDD',dd
  
#     print 'notbranched',NotBranched
    lenNotbranched=len(NotBranched)
   
    
    for t in range(lenNotbranched):  
        for i in range (VarNum):
            sumlist=list()
            if t>0:
                for p in range(t):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                        
            for j in range(ConstNum):
                sumlist.append(dd[t*ConstNum+j,i+VarNum])
            if i in NotBranched and i > NotBranched[t]:
                indx=NotBranched.index(i)
                indy=NotBranched.index(NotBranched[t])
                
                for x in range(indx-indy-1):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                for j in range(ConstNum):
                    sumlist.append(dd[(indx)*ConstNum+j,NotBranched[t]+VarNum])
                    dd[(indx)*ConstNum+j,NotBranched[t]+VarNum]=0
#                 sumarray.append([t+indx,indy])
                if t<lenNotbranched-2 :
                    for p in range(lenNotbranched-indx-1):
                        for j in range (ConstNum):
                            sumlist.append(0)
            else:
                if t<lenNotbranched-1 :
#                     print 'lenNotbranched',lenNotbranched
                    for p in range(lenNotbranched-t-1):
                            for j in range (ConstNum):
                                sumlist.append(0) 
#             print 't,i',t,',',i
#             print 'sumlist p2',sumlist
#             print 'lensumlist p2',len(sumlist)
            sumarray.append(sumlist)
#     print 'part2' ,sumarray  
    
    rows=[]
    for j in range(numrows):
        my_row.append(1)
    rows.append([range(numrows),my_row])  
    def populatebyrow(SecProb):
        SecProb.objective.set_sense(SecProb.objective.sense.maximize)
        SecProb.variables.add(obj = my_obj,lb=my_lb, names = my_colnames)
        
        for i in range (len(sumarray)):
            if all(k == 0 for k in sumarray[i])!=True:
#                 print 'i',i
                sumline=[]
                sumline.append([range(len(sumarray[i])),sumarray[i]])
#                 print 'sumline',sumline
                SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E",
                                rhs = [0], names = ["u"+str(i)])
        SecProb.linear_constraints.add(lin_expr = rows, senses = "L",
                                             rhs = [1], names = ["t0"])
    
    my_SecProb = CPX.Cplex()
    handle = populatebyrow(my_SecProb)
    my_SecProb.solve()
    my_SecProb.write("lpex2.lp")
    SecVariables    = my_SecProb.solution.get_values()
    SecObjective=my_SecProb.solution.get_objective_value()
#     print 'SecVariables,SecObjective,my_colnames',SecVariables, SecObjective,my_colnames
    return SecVariables, SecObjective,my_colnames

In [3]:
def GeneratintCutParameters ( m, n, my_rhs, my_coef, BranchedVariables,SecVariables): 
    BranchedLen=len(BranchedVariables)
    unbranched=list()
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)  
    LeUnbranched=len(unbranched)                   
#     NodeNum=n-len(unbranched)
#     k=NodeNum-1 
    lhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),BranchedLen)))
    rhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),1)))

    for i in range (LeUnbranched):   
        for h in range (m):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+m+h][BranchedVariables.index(j)]=my_coef[h][j]*SecVariables[i*(2*m+4*n)+m+h]  
                rhs[i*(2*m+4*n)+m+h]=my_rhs[h]*SecVariables[i*(2*m+4*n)+m+h]  
            
        for h in range (n):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+2*m+2*n+j][BranchedVariables.index(j)]=SecVariables[i*(2*m+4*n)+2*m+2*n+j]  
                rhs[i*(2*m+4*n)+2*m+2*n+h]=SecVariables[i*(2*m+4*n)+2*m+2*n+h]  
            
#         for h in range (n):
        for j in (BranchedVariables):
            lhs[i*(2*m+4*n)+2*m+3*n+j][BranchedVariables.index(j)]=-1*SecVariables[i*(2*m+4*n)+2*m+3*n+j]  

    #     print 'lefttt',lhs,'rightttt', rhs
    LenLeft=len(lhs)
    LenRight=len(rhs)
    LenSec=len(SecVariables)
#     print 'LenLeft',LenLeft
#     print'LenRight',LenRight
#     print 'LenSec',LenSec
#     indicesLhs=[]
#     for j in range (8):
#         indicesLhs.append([i for i in range(LenLeft) if lhs[i][j]!=0]) 
#     print indicesLhs
#     indicesRhs = [(i,rhs[i]) for i in range(LenRight) if rhs[i]>0]
#     print 'indicesRhs',indicesRhs
#     indiceSecVariables=[(i , SecVariables[i]) for i in range(LenSec) if SecVariables[i]!=0]
#     ValueSecVariables=[SecVariables[i] for i in range(LenSec) if SecVariables[i]!=0]
#     print 'positiveLeftHand',indicesLhs 
#     print 'indiceSecVariables',indiceSecVariables
#     print 'ValueSecVariables', ValueSecVariables
    FinalLhs=np.sum(lhs, axis=0)
    FinalRhs=np.sum(rhs, axis=0)
    print 'FinalLhs',FinalLhs, 'FinalRhs',FinalRhs
    return FinalLhs,FinalRhs



In [4]:
class MyCut(CPX_CB.UserCutCallback):

    def __call__(self):
         
        self.times_called += 1
        print  'self.times_called', self.times_called
        branched=list()
        values=list()
        branchedNames=list()
        global BranchedMatrix
        senses= "L"
#         print 'self.get_lower_bounds(s)',self.get_lower_bounds()
#         print 'self.get_upper_bounds(s)',self.get_upper_bounds()
#         print 'objective value', self.get_objective_value()
#         print 'values', self.get_values()
        print 'get_num_nodes', self.get_num_nodes()

        lb=self.get_lower_bounds()
        ub=self.get_upper_bounds()               
        for j in range(len(lb)):
            if lb[j]==ub[j]:
                branched.append(j)
                values.append(lb[j])
                branchedNames.append(VarsName[j])
                
        print 'values',values
        print 'branchedNames',branchedNames
        print 'branched',branched

        if len(values)>0:  
            BranchedMatrix.append([branched,values])
#             BranchedValue.append(values)
#             print 'BranchedMatrix',BranchedMatrix

            if [BranchedMatrix[-1]]!=[BranchedMatrix[-2]] :
                SecModelParameterOutput=SecModelParameter(values, self.m, self.n, self.my_rhs, self.my_coef,branched)
                SecondModelOutput=SecondModel (SecModelParameterOutput[0],SecModelParameterOutput[1],SecModelParameterOutput[2])
                print 'SecondModelOutput[1]',SecondModelOutput[1]
                print 'SecondModelOutput[0]',SecondModelOutput[0]
                if SecondModelOutput[1]>0.000001:
    #                 print 'bayad cut bezane'
                    output=GeneratintCutParameters (self.m, self.n, self.my_rhs, self.my_coef,branched, SecondModelOutput[0])
                    print 'output',output[0]
                    print 'output[1]',output[1]
                    lhs=[CPX.SparsePair(ind =branchedNames, val = output[0])]
                    print 'lhs',lhs[0]
                    self.add(cut=lhs[0],rhs=output[1][0],sense=senses)
                    print 'cut added'



In [5]:
def admipex3(c):
    
    c.parameters.preprocessing.presolve = 0

    c.parameters.preprocessing.reduce = 0

    c.parameters.preprocessing.linear.set(0)

    c.parameters.preprocessing.numpass = 0

    c.parameters.mip.strategy.presolvenode = -1

    c.parameters.preprocessing.linear = 0
    
def CutCplex(c):
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    
c = CPX.Cplex('NewExample.lp')
m=c.linear_constraints.get_num()
n=c.variables.get_num()
my_rhs=[]
my_coef=[]
for i in range(m):
    my_row=[]
    for j in range(n):
        my_row.append(c.linear_constraints.get_coefficients(i,j))
    my_coef.append(my_row)

for i in range (m):
    my_rhs.append(c.linear_constraints.get_rhs(i))

# admipex3(c)
# CutCplex(c)
VarsName=c.variables.get_names()
c.set_log_stream(sys.stdout)
c.set_results_stream(sys.stdout)

cutinst=c.register_callback(MyCut)
cutinst.times_called = 0
cutinst.m=m
cutinst.n=n
cutinst.my_rhs=my_rhs
cutinst.my_coef=my_coef
cutinst.VarsName=VarsName
BranchedMatrix=[[[-1], [-0.0]]]
c.parameters.mip.interval.set(1)
c.parameters.preprocessing.linear=0
c.parameters.mip.strategy.search.set(c.parameters.mip.strategy.search.values.traditional)
c.solve()

c.write("lpex1.lp")
solution = c.solution

# the following line prints the corresponding string
print solution.status[solution.get_status()]
print "Objective value = " , solution.get_objective_value()
print
x = solution.get_values(0, c.variables.get_num()-1)
for j in range(c.variables.get_num()):
    if fabs(x[j]) > 1.0e-10:
        print "Column %d: Value = %17.10g" % (j, x[j])

print "Branch callback was called ", cutinst.times_called, "times"


CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Interval                            1
CPXPARAM_MIP_Strategy_Search                     1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 30 rows, 30 columns, and 896 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.24 ticks)
Probing time = 0.00 sec. (0.04 ticks)
Tried aggregator 1 time.
Reduced MIP has 30 rows, 30 columns, and 896 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.35 ticks)
Probing time = 0.00 sec. (0.04 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.16 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap        

      1     3     1401.6753     7     1237.0000     1401.1404       78   13.27%             x19 D      1      0      1
self.times_called 15
get_num_nodes 2
values [1.0, -0.0]
branchedNames ['x4', 'x19']
branched [4, 19]
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 1568 columns.
Reduced LP has 463 rows, 3472 columns, and 79642 nonzeros.
Presolve time = 0.02 sec. (7.49 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =            54.581392
Iteration:    67   Scaled dual infeas =            22.717461
Iteration:   129   Scaled dual infeas =            22.717461
Iteration:   191   Scaled dual infeas =            22.717461
Perturbation started.
Iteration:   202   Scaled dual infeas =            22.717465
Iteration:   264   Scaled dual infeas =            16.075302
Iteration:   326   Scaled dual i

      2     4     1398.8303     8     1237.0000     1398.7760       87   13.08%              x4 U      2      1      2
self.times_called 17
get_num_nodes 3
values [1.0, -0.0, 1.0]
branchedNames ['x4', 'x19', 'x24']
branched [4, 19, 24]
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 1458 columns.
Reduced LP has 460 rows, 3402 columns, and 76096 nonzeros.
Presolve time = 0.02 sec. (7.16 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =            51.272724
Iteration:    65   Scaled dual infeas =            30.204138
Iteration:   127   Scaled dual infeas =            30.204138
Iteration:   189   Scaled dual infeas =            30.204138
Perturbation started.
Iteration:   202   Scaled dual infeas =            30.204145
Iteration:   264   Scaled dual infeas =            14.494352
Iteration:   326

      3     5     1396.4067     6     1237.0000     1396.1314       92   12.86%             x24 U      3      2      3
self.times_called 19
get_num_nodes 4
values [1.0, 1.0, -0.0, 1.0]
branchedNames ['x4', 'x15', 'x19', 'x24']
branched [4, 15, 19, 24]
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 1352 columns.
Reduced LP has 456 rows, 3328 columns, and 72602 nonzeros.
Presolve time = 0.01 sec. (6.84 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =           402.880405
Iteration:    65   Scaled dual infeas =            43.722283
Iteration:   127   Scaled dual infeas =            43.722283
Iteration:   189   Scaled dual infeas =            43.722283
Perturbation started.
Iteration:   202   Scaled dual infeas =            43.722344
Iteration:   264   Scaled dual infeas =            24.704037


      4     6     1396.1314     6     1237.0000     1394.4400       93   12.73%             x15 U      4      3      4
self.times_called 21
get_num_nodes 5
values [1.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x4', 'x8', 'x15', 'x19', 'x24']
branched [4, 8, 15, 19, 24]
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 1250 columns.
Reduced LP has 451 rows, 3250 columns, and 69160 nonzeros.
Presolve time = 0.02 sec. (6.52 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =           682.586204
Iteration:    63   Scaled dual infeas =            63.752486
Iteration:   125   Scaled dual infeas =            63.752486
Iteration:   187   Scaled dual infeas =            53.891015
Iteration:   249   Scaled dual infeas =             6.604090
Iteration:   311   Scaled dual infeas =             6.477203
Iteratio

...

    194    20        cutoff           1317.0000     1332.4381      953    1.17%              x5 U    194     70     11
    195    19        cutoff           1317.0000     1329.5558      958    0.95%             x23 D    195     49      9
    196    18    infeasible           1317.0000     1328.3727      958    0.86%             x15 D    196    116      7
    197    17        cutoff           1317.0000     1327.8436      965    0.82%             x14 D    197     57      7
self.times_called 298
get_num_nodes 198
values [1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x2', 'x7', 'x11', 'x12', 'x14', 'x16', 'x17', 'x18', 'x19', 'x22', 'x23', 'x24', 'x29']
branched [2, 7, 11, 12, 14, 16, 17, 18, 19, 22, 23, 24, 29]
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 578 columns.
Reduced LP has 375 rows, 2482 columns, a

    198    16        cutoff           1317.0000     1327.3694      971    0.79%             x12 U    198     89      7
    199    15        cutoff           1317.0000     1327.2273      974    0.78%             x13 D    199    131      5
    200    14        cutoff           1317.0000     1327.1350      976    0.77%              x8 D    200    177      6
Elapsed time = 192.55 sec. (32.25 ticks, tree = 0.01 MB, solutions = 5)
self.times_called 299
get_num_nodes 201
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x2', 'x4', 'x7', 'x11', 'x13', 'x14', 'x15', 'x17', 'x19', 'x22', 'x23', 'x24', 'x25', 'x26', 'x28']
branched [2, 4, 7, 11, 13, 14, 15, 17, 19, 22, 23, 24, 25, 26, 28]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 450 columns.
Reduced LP has 346 rows, 2250 columns, and 37624 nonzeros.
Presolve time = 0.02 sec. (3.57 ticks)
Initializing dual steep norms . . .

Ite

    201    13        cutoff           1317.0000     1326.9853      980    0.76%             x22 U    201     33      7
    202    12        cutoff           1317.0000     1325.4505      982    0.64%             x28 D    202     31      5
    203    11        cutoff           1317.0000     1325.3101      984    0.63%              x3 D    203     22      8
    204    10    infeasible           1317.0000     1324.7101      984    0.59%              x7 D    204    178      7
    205     9        cutoff           1317.0000     1323.1553      988    0.47%             x24 D    205     88      6
    206     8    infeasible           1317.0000     1322.9758      988    0.45%              x7 D    206    117      8
    207     7    infeasible           1317.0000     1322.5938      988    0.42%             x24 D    207    137      8
self.times_called 300
get_num_nodes 208
values [1.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x2', 'x3', 'x7', '

    208     6        cutoff           1317.0000     1322.5842      989    0.42%             x15 D    208     90      8
    209     5    infeasible           1317.0000     1321.4710      989    0.34%              x5 U    209    157      7
    210     4    infeasible           1317.0000     1320.1297      989    0.24%             x24 D    210     81      8
Elapsed time = 194.84 sec. (33.29 ticks, tree = 0.01 MB, solutions = 5)
self.times_called 301
get_num_nodes 211
values [1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x2', 'x4', 'x5', 'x7', 'x8', 'x11', 'x13', 'x14', 'x15', 'x17', 'x18', 'x19', 'x23', 'x24', 'x25', 'x26', 'x29']
branched [2, 4, 5, 7, 8, 11, 13, 14, 15, 17, 18, 19, 23, 24, 25, 26, 29]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 338 columns.
Reduced LP has 313 rows, 2002 columns, and 31934 nonzeros.
Presolve time = 0.00 sec. (3.03 ticks)
Initializing

    211     3        cutoff           1317.0000     1319.3128      991    0.18%              x5 U    211     58      8
    212     2        cutoff           1317.0000     1318.6885      993    0.13%             x14 D    212    136      7
    213     1        cutoff           1317.0000     1318.5998      994    0.12%             x18 D    213     79      6
    214     0        cutoff           1317.0000     1317.0000      996    0.00%             x13 D    214     78      5

Cover cuts applied:  41
Mixed integer rounding cuts applied:  3
Zero-half cuts applied:  6
Gomory fractional cuts applied:  1
User cuts applied:  20

Root node processing (before b&c):
  Real time             =    0.22 sec. (10.43 ticks)
Sequential b&c:
  Real time             =  195.72 sec. (23.41 ticks)
                          ------------
Total (root+branch&cut) =  195.94 sec. (33.84 ticks)
MIP_optimal
Objective value =  1317.0

Column 1: Value =                 1
Column 2: Value =                 1
Column 3: Val